In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
 # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print('----------START GPU----------')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
            print('----------END GPU----------')
    except RuntimeError as e:
 # Memory growth must be set before GPUs have been initialized
        print(e)

# Import some packages

You should load packages you may need in this project first

In [ ]:
import skimage.io
import skimage.color
import skimage.transform
import matplotlib.pyplot as plt


import numpy as np
import os
import random
import cv2
import pandas as pd

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D,BatchNormalization
from keras.layers import Activation
from keras import backend as K

#Sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix

# Load character data from the dataset subfolder


In [ ]:
#Initialize a list of paths for images
imagepaths = []
import os
for dirname, _, filenames in os.walk('1125data'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        #print(os.path.join(dirname, filename))
        if path.endswith("png"):
            imagepaths.append(path)
            
print(len(imagepaths))

#print(imagepaths)

In [ ]:
# #Defining a function that plots the image selected from a path

# def img_plot(img_path):
#     img = cv2.imread(img_path)
#     #convert to RGB space
#     img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     image_grayscale_blurred = cv2.GaussianBlur(img_rgb, (15,15), 0)
#     im3 =  cv2.resize(image_grayscale_blurred, (150,150), interpolation = cv2.INTER_AREA)  
#  #ar = np.array(resized_img)
#  #ar = resized_img.reshape(1,784) 
     
# #check the shape of the image
#     print("Shape of the image is ", im3.shape)
#     print(img_rgb.shape)
    

In [ ]:
# img_plot(imagepaths[0])

In [ ]:

#skimage.io.imread()
#skimage.color.rgb2gray()
data_dir = './1125data/'
classes = os.listdir(data_dir)
data = []
for cls in classes:
    files = os.listdir(data_dir+cls)
    for f in files:        
        img = skimage.io.imread(data_dir+cls+"/"+f)
#         img_rgb = skimage.color.rgb2gray(img)
#         image_grayscale_blurred = cv2.GaussianBlur(img_rgb, (15,15), 0)
        img =cv2.resize(img,(224, 224))

        data.append({
            'x':img,
            'y':cls
        })
        
random.shuffle(data)




In [ ]:
len(data)

# create train data set and test data set 
Using 80/20 rule, 80 precent for trainning, and 20 percent for testing.

In [ ]:
#x_train =
#y_train =

#x_test =
#y_test =
X = [d['x'] for d in data]
y = [d['y'] for d in data]

y_dic = list(np.unique(y))
y = [y_dic.index(v) for v in y]
x_train = np.array(X[:int(len(X)*0.8)])
y_train = np.array(y[:int(len(X)*0.8)])

x_test = np.array(X[int(len(X)*0.8):])
y_test = np.array(y[int(len(X)*0.8):])

# Examming and Understand data set for trainning


In [ ]:
# # you may print out some infomation about train data set 
# print(x_train[0])
# # print(y_train)
# print(len(y_train))

# Decide a few parameters for trainning

such as batch size, epochs, image size in rows and colomns, etc.

In [ ]:
batch_size = 32#batchsize：批大小。在深度学习中，一般采用SGD训练，即每次训练在训练集中取batchsize个样本训练；

num_classes = len(classes)
epochs =  10

## input image dimensions
img_rows, img_cols =224,224

num_classes

# Data normalization

In [ ]:
#keras.utils.to_categorical()
def extend_channel(data):
    if K.image_data_format() == 'channels_first':
        data = data.reshape(data.shape[0], 1, img_rows, img_cols)
    else:
        data = data.reshape(data.shape[0], img_rows, img_cols, 1)
        
    return data

x_train = extend_channel(x_train)
x_test = extend_channel(x_test)

input_shape = x_train.shape[1:]


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train2 = keras.utils.to_categorical(y_train, num_classes)
y_test2 = keras.utils.to_categorical(y_test, num_classes)
input_shape

np.set_printoptions(threshold=np.inf)

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
# datagen =ImageDataGenerator(
#         featurewise_center=False,  # set input mean to 0 over the dataset 布尔值，使输入数据集去中心化（均值为0）, 按feature执行
#         samplewise_center=False,  # set each sample mean to 0 布尔值，使输入数据的每个样本均值为0 
#         featurewise_std_normalization=False,  # divide inputs by std of the dataset布尔值，将输入除以数据集的标准差以完成标准化, 按feature执行。
#         samplewise_std_normalization=False,  # divide each input by its std布尔值，将输入的每个样本除以其自身的标准差
#         zca_whitening=False,  # apply ZCA whitening布尔值，对输入数据施加ZCA白化
#         rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
#         zoom_range = 0.1, # Randomly zoom image 
#         width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
#         height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
#         horizontal_flip=False,  # randomly flip images
#         vertical_flip=False)  # randomly flip images
# datagen.fit(x_train)


# Create a CNN model for character recognition

This is an important part of this project. You have to create a cnn model using tensorflow and keras to train the data.


In [ ]:
from keras.callbacks import ReduceLROnPlateau

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.5, min_lr=0.0001)
# learning_rate_reduction = ReduceLROnPlateau(monitor='val_loss', patience = 2, verbose=1,factor=0.5, min_lr=0.001)

In [ ]:
#Alexnet_our version
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same',activation ='relu', input_shape = (224,224,1)))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2, padding = 'Same'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2, padding = 'Same'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same',activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides = 2, padding = 'Same'))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(Dense(128))
model.add(Dropout(0.3))
model.add(Dense(10, activation = "softmax"))

#AlexNet_original
# model = Sequential()
# model.add(Conv2D(96,(11,11),strides=(4,4),input_shape=(227,227,3),padding='valid',activation='relu',kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
# model.add(Conv2D(256,(5,5),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
# model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
# model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
# model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))
# model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))
# model.add(Flatten())
# model.add(Dense(4096,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(4096,activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1000,activation='softmax'))

#Z_net
# model = Sequential()  
# model.add(Conv2D(96,(7,7),strides=(2,2),input_shape=(224,224,1),padding='valid',activation='relu',kernel_initializer='uniform'))  
# model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
# model.add(Conv2D(256,(5,5),strides=(2,2),padding='same',activation='relu',kernel_initializer='uniform'))  
# model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
# model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
# model.add(Conv2D(384,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
# model.add(Conv2D(256,(3,3),strides=(1,1),padding='same',activation='relu',kernel_initializer='uniform'))  
# model.add(MaxPooling2D(pool_size=(3,3),strides=(2,2)))  
# model.add(Flatten())  
# model.add(Dense(4096,activation='relu'))  
# model.add(Dropout(0.5))  
# model.add(Dense(4096,activation='relu'))  
# model.add(Dropout(0.5))  
# model.add(Dense(1000,activation='softmax')) 

model.compile(optimizer = 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

# Compile the model

In [ ]:
#model.compile()
# #model.summary()

import tensorflow as tf
# 必须要下面这行代码
tf.compat.v1.disable_eager_execution()
print(tf.__version__)
 
# 我自己使用的函数
def get_flops_params():
    sess = tf.compat.v1.Session()
    graph = sess.graph
    flops = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation())
    params = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    print('FLOPs: {};    Trainable params: {}'.format(flops.total_float_ops, params.total_parameters))
 
 
# 网上推荐的
# sess = tf.compat.v1.Session()
# graph = sess.graph
# stats_graph(graph)
def stats_graph(graph):
    flops = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.float_operation())
    # print('FLOPs: {}'.format(flops.total_float_ops))
    params = tf.compat.v1.profiler.profile(graph, options=tf.compat.v1.profiler.ProfileOptionBuilder.trainable_variables_parameter())
    # print('Trainable params: {}'.format(params.total_parameters))
    print('FLOPs: {};    Trainable params: {}'.format(flops.total_float_ops, params.total_parameters))
 
 
def get_flops(model):
    run_meta = tf.compat.v1.RunMetadata()
    opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
    # We use the Keras session graph in the call to the profiler.
    flops = tf.compat.v1.profiler.profile(graph=tf.compat.v1.keras.backend.get_session().graph, run_meta=run_meta, cmd='op', options=opts)
    return flops.total_float_ops  # Prints the "flops" of the model.
 
# 必须使用tensorflow中的keras才能够获取到FLOPs, 模型中的各个函数都必须使用tensorflow.keras中的函数，和keras混用会报错

# 获取模型每一层的参数详情
model.summary()
# 获取模型浮点运算总次数和模型的总参数
get_flops_params()

In [ ]:
import keras
def time_per_layer(model):
    new_model = model

    times = np.zeros((len(model.layers), 2))
    inp = np.ones((224, 224, 1))

    for i in range(1, len(model.layers)):
        new_model = keras.models.Model(inputs=[model.input], outputs=[model.layers[-i].output])

        # new_model.summary()
        new_model.predict(inp[None, :, :, :])

        t_s = time.time()
        new_model.predict(inp[None, :, :, :])
        t_e2 = time.time() - t_s

        times[i, 1] = t_e2
        del new_model

    for i in range(0, len(model.layers) - 1):
        times[i, 0] = abs(times[i + 1, 1] - times[i, 1])

    times[-1, 0] = times[-1, 1]


    return times

# from net_flops import net_flops

# net_flops(model,table=True)
import time
import matplotlib.pyplot as plt
times = time_per_layer(model)

plt.style.use('ggplot')
x = [model.layers[-i].name for i in range(1,len(model.layers))]
#x = [i for i in range(1,len(model.layers))]
g = [times[i,0] for i in range(1,len(times))]
x_pos = np.arange(len(x))
plt.bar(x, g, color='#7ed6df')
plt.xlabel("Layers")
plt.ylabel("Processing Time")
plt.title("Processing Time of each Layer")
plt.xticks(x_pos, x,rotation=90)
print('Sum of all the layers time(s):')
print( sum(g))
plt.show()

In [ ]:
from net_flops import net_flops
# from time_per_layer import time_per_layer
net_flops(model,table=True)

# Train your model


In [ ]:
# #model.fit()
# history=model.fit(datagen.flow(x_train, y_train2,batch_size=batch_size),
#           epochs=20,
#           verbose=1,
#           validation_data=(x_test, y_test2))

history=model.fit(x_train, y_train2,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test2))

# from keras.callbacks import EarlyStopping
# early_stopping = EarlyStopping(monitor='val_loss', patience=50, verbose=2)
# # 训练
# history = model.fit(x_train, y_train2, epochs=20, batch_size=128, validation_data=(x_test, y_test2), verbose=2, shuffle=False, callbacks=[early_stopping])
#monitor: 需要监视的量，val_loss or val_acc
#patience: 当early stop被激活(如发现loss相比上一个epoch训练没有下降)，则经过patience个epoch后停止训练
#verbose: 信息展示模式


# Model Evaluation 

Evaluate your model with test dataset.

make sure your model accuracy should be more than 99 percent. Otherwise, you have to go back to fine tune your model with whatever methods you have learned to improve its accuracy. You have to handle over-fitting or under-fitting problem you may encounter.

In [ ]:
#score = model.evaluate(xxxxxxx)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
score = model.evaluate(x_test, y_test2, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

# Save your model weights

You should save your model for car plate recognition purpose later.


In [ ]:
model.save_weights('weights_train_model1129alexnet1.h5')
model.save('train_model1129alexnet1.h5')

In [ ]:
#plt.figure(figsize=(8, 8))
#plt.subplot(1, 2, 1)
#plt.plot(epochs_range, acc, label='Training Accuracy')
#plt.plot(epochs_range, val_acc, label='Validation Accuracy')
#plt.legend(loc='lower right')
#plt.title('Training and Validation Accuracy')

#plt.subplot(1, 2, 2)
#plt.plot(epochs_range, loss, label='Training Loss')
#plt.plot(epochs_range, val_loss, label='Validation Loss')
#plt.legend(loc='upper right')
#plt.title('Training and Validation Loss')
#plt.show()
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs_range = range(1, len(loss) + 1)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']


plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

plt.savefig("1129Alexnet1.png")